In [1]:
import os
import math
import argparse
from tensorflow_asr.utils import env_util
#libraries
import os
import pandas as pd
import numpy as np
from keras import layers, models, optimizers
from keras import backend as K
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPooling2D, Dropout,concatenate
from keras.layers import Dense, Flatten,Input, LSTM, Conv2D, MaxPooling1D, Dropout, Activation, Bidirectional,Dense, Input, Embedding, Dropout,SpatialDropout1D
from keras.optimizers import Adam
from keras.layers.embeddings import Embedding
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from keras.engine import Layer
from keras.layers import Activation, Add, Bidirectional, Conv1D, Dense, Dropout, Embedding, Flatten
from keras.layers import concatenate, GRU, Input, LSTM, MaxPooling1D
from keras.layers import GlobalAveragePooling1D,  GlobalMaxPooling1D, SpatialDropout1D
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing import text, sequence
from sklearn.metrics import accuracy_score, roc_auc_score, log_loss
from sklearn.model_selection import train_test_split
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras.utils import to_categorical
import matplotlib.pyplot as plt
#from utils import combine_images
from keras.layers.core import Reshape, Flatten
from PIL import Image
#from capsulelayers import CapsuleLayer, PrimaryCap, Length, Mask
from keras.layers.normalization import BatchNormalization

env_util.setup_environment()
import tensorflow as tf

DEFAULT_YAML = "/Users/vaibhavsingh/Desktop/TensorFlowASR/examples/contextnet/configs_local/config_macbook_sanity.yml"

import math
import os

from tensorflow_asr.landescapeviz import visualizer
from tensorflow_asr.landescapeviz import utils
from tensorflow_asr.landescapeviz import trajectory
from tensorflow_asr.configs.config import Config
from tensorflow_asr.datasets.asr_dataset import ASRTFRecordDataset, ASRSliceDataset
from tensorflow_asr.featurizers.speech_featurizers import TFSpeechFeaturizer
from tensorflow_asr.featurizers.text_featurizers import SubwordFeaturizer, SentencePieceFeaturizer, CharFeaturizer
from tensorflow_asr.models.transducer.contextnet import ContextNet
from tensorflow_asr.utils import app_util
from tensorflow_asr.optimizers.schedules import TransformerSchedule
from tqdm import tqdm
import argparse
from tensorflow_asr.utils import env_util, file_util

env_util.setup_environment()
import tensorflow as tf
tf.keras.backend.clear_session()
tf.config.optimizer.set_experimental_options({"auto_mixed_precision": False})
strategy = env_util.setup_strategy([0])

config = Config(DEFAULT_YAML)
speech_featurizer = TFSpeechFeaturizer(config.speech_config)

Use RNNT loss in TensorFlow


In [2]:
text_featurizer = CharFeaturizer(config.decoder_config)
tf.random.set_seed(0)

test_dataset = ASRSliceDataset(
        speech_featurizer=speech_featurizer,
        text_featurizer=text_featurizer,
        **vars(config.learning_config.test_dataset_config)
    )


In [3]:
batch_size = config.learning_config.running_config.batch_size
test_data_loader = test_dataset.create(batch_size)
contextnet = ContextNet(**config.model_config, vocabulary_size=text_featurizer.num_classes)
contextnet.make(speech_featurizer.shape)
#TODO= contextnext.add(lambda layer)
#input = out of contextnext 
#out = tf.l2norm 
contextnet.load_weights("/Users/vaibhavsingh/Desktop/TensorFlowASR/examples/contextnet/checkpoint/model_trained_01.h5", by_name=True)  


Reading /Users/vaibhavsingh/Desktop/TensorFlowASR/scripts/Datasets/test-clean-sanity/transcripts.tsv ...
================INSIDE TRANSDUCER=================
before going to the encoder================= {'inputs': <tf.Tensor 'Placeholder:0' shape=(None, None, 80, 1) dtype=float32>, 'inputs_length': <tf.Tensor 'Placeholder_1:0' shape=(None,) dtype=int32>, 'signal': <tf.Tensor 'Placeholder_4:0' shape=(None, None, 1) dtype=float32>, 'predictions': <tf.Tensor 'Placeholder_2:0' shape=(None, None) dtype=int32>, 'predictions_length': <tf.Tensor 'Placeholder_3:0' shape=(None,) dtype=int32>}
inside the call fx of encoder------------------------------ [<tf.Tensor 'Placeholder:0' shape=(None, None, 80, 1) dtype=float32>, <tf.Tensor 'Placeholder_1:0' shape=(None,) dtype=int32>, <tf.Tensor 'Placeholder_4:0' shape=(None, None, 1) dtype=float32>]
Inside convBlock======== Tensor("contextnet/contextnet_encoder/contextnet_encoder_reshape/Reshape:0", shape=(None, None, 80), dtype=float32)

Inside convBlock

In [6]:
with tf.GradientTape(persistent=True) as tape:
    
#     batch_size = config.learning_config.running_config.batch_size
#     contextnet = ContextNet(**config.model_config, vocabulary_size=text_featurizer.num_classes)
#     contextnet.make(speech_featurizer.shape)
#     contextnet.load_weights("/Users/vaibhavsingh/Desktop/TensorFlowASR/examples/contextnet/checkpoint/model_trained_01.h5", by_name=True)  

    # contextnet.load_weights("model_all_re_cn_09.h5", by_name=True)

#     contextnet.summary(line_length=100)
    contextnet.add_featurizers(speech_featurizer, text_featurizer)
    optimizer = tf.keras.optimizers.Adam(
        TransformerSchedule(
            d_model=contextnet.dmodel,
            warmup_steps=config.learning_config.optimizer_config.pop("warmup_steps", 10000),
            max_lr=(0.05 / math.sqrt(contextnet.dmodel))
        ),
        **config.learning_config.optimizer_config
    )

    contextnet.compile(
        optimizer=optimizer,
        experimental_steps_per_execution=1,
        global_batch_size=1,
        blank=text_featurizer.blank
    )
    encoder = contextnet.layers[0]

   





    
    for i, j in test_data_loader:

        inputs = tf.Variable(i["inputs"])
        inputs_length = tf.Variable(i["inputs_length"])
        signal = tf.Variable(i["signal"])

        encoder_output = encoder.call_feature_output([inputs, inputs_length, signal])
        print("encoder output--------", encoder_output)
        individual_layer_output_list = encoder.feature_output
        print("individual output===========", individual_layer_output_list)
    
        #Basically this output is same as the input, will be used to check the grad with input, should be equal to 1
        out_1_layer = tf.Variable(individual_layer_output_list[0])
        #make this 
        norm_raw_output = tf.norm(encoder_output, axis=1)
        
#         final_node = tf.math.argmax(tf.squeeze(tf.reduce_sum(encoder_output, axis=1)))
#         print("fial_node====", final_node)
        k = tape.gradient(encoder_output, inputs)
        
#         print("-----xx", inputs.sh)
        print("k---", k)
        
        #Reseting these layers after processing.
        encoder.feature_output = []
       
        tape.reset()
        break
        
        
    


  opt = tf.keras.mixed_precision.experimental.LossScaleOptimizer(opt)
inside the call fx of encoder------------------------------ [<tf.Variable 'Variable:0' shape=(1, 608, 80, 1) dtype=float32, numpy=
array([[[[-0.88430405],
         [-1.0025831 ],
         [-1.8104411 ],
         ...,
         [-1.1646435 ],
         [-1.4017149 ],
         [-2.1446335 ]],

        [[-0.95217824],
         [-0.8431829 ],
         [-0.7029175 ],
         ...,
         [-0.9792209 ],
         [-1.6376293 ],
         [-2.191236  ]],

        [[-1.1578585 ],
         [-0.9471249 ],
         [-0.73311234],
         ...,
         [-1.1570454 ],
         [-1.768073  ],
         [-2.343823  ]],

        ...,

        [[-0.83737606],
         [-0.749652  ],
         [-0.6308468 ],
         ...,
         [-1.1891571 ],
         [-1.4152755 ],
         [-1.9534897 ]],

        [[-1.0136111 ],
         [-0.8318222 ],
         [-0.63500834],
         ...,
         [-1.0849377 ],
         [-1.2815615 ],
         [-

k--- tf.Tensor(
[[[[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  ...

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]]], shape=(1, 608, 80, 1), dtype=float32)


In [25]:
# print(adc[0])
# print(inputs)
nb = np.array([1,2,3,4])
nb = tf.convert_to_tensor(nb)
print(nb)
m = tf.math.argmax(nb)
print(m)

tf.Tensor([1 2 3 4], shape=(4,), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)


In [20]:
print(inputs)

<tf.Variable 'Variable:0' shape=(1, 608, 80, 1) dtype=float32, numpy=
array([[[[-0.88430405],
         [-1.0025831 ],
         [-1.8104411 ],
         ...,
         [-1.1646435 ],
         [-1.4017149 ],
         [-2.1446335 ]],

        [[-0.95217824],
         [-0.8431829 ],
         [-0.7029175 ],
         ...,
         [-0.9792209 ],
         [-1.6376293 ],
         [-2.191236  ]],

        [[-1.1578585 ],
         [-0.9471249 ],
         [-0.73311234],
         ...,
         [-1.1570454 ],
         [-1.768073  ],
         [-2.343823  ]],

        ...,

        [[-0.83737606],
         [-0.749652  ],
         [-0.6308468 ],
         ...,
         [-1.1891571 ],
         [-1.4152755 ],
         [-1.9534897 ]],

        [[-1.0136111 ],
         [-0.8318222 ],
         [-0.63500834],
         ...,
         [-1.0849377 ],
         [-1.2815615 ],
         [-2.2296257 ]],

        [[-0.91157675],
         [-0.7796618 ],
         [-0.61910915],
         ...,
         [-1.1070294 ],
      

In [231]:
for i, j in test_data_loader:
    print(i)
    break

{'inputs': <tf.Tensor: shape=(1, 608, 80, 1), dtype=float32, numpy=
array([[[[-0.88430405],
         [-1.0025831 ],
         [-1.8104411 ],
         ...,
         [-1.1646435 ],
         [-1.4017149 ],
         [-2.1446335 ]],

        [[-0.95217824],
         [-0.8431829 ],
         [-0.7029175 ],
         ...,
         [-0.9792209 ],
         [-1.6376293 ],
         [-2.191236  ]],

        [[-1.1578585 ],
         [-0.9471249 ],
         [-0.73311234],
         ...,
         [-1.1570454 ],
         [-1.768073  ],
         [-2.343823  ]],

        ...,

        [[-0.83737606],
         [-0.749652  ],
         [-0.6308468 ],
         ...,
         [-1.1891571 ],
         [-1.4152755 ],
         [-1.9534897 ]],

        [[-1.0136111 ],
         [-0.8318222 ],
         [-0.63500834],
         ...,
         [-1.0849377 ],
         [-1.2815615 ],
         [-2.2296257 ]],

        [[-0.91157675],
         [-0.7796618 ],
         [-0.61910915],
         ...,
         [-1.1070294 ],
        

In [22]:
model=Sequential()
model.add(Dense(40, activation='softmax',input_dim=12))
model.add(Dense(60, activation='relu'))
model.add(Dense(units=6, activation='relu'))
model.add(Dense(units=3, activation='relu'))

opt=tf.keras.optimizers.Adam(lr=0.001)


x=np.random.random(12).reshape(-1,12)
with tf.GradientTape(persistent=True) as tape:
    y = tf.convert_to_tensor(x)
    tape.watch(y)
    print(y.shape)
    x2 = model([y])
    print(x2.shape, x2)
    loss = -tf.reduce_mean(x2)
    print(loss)
    
grad = tape.gradient(x2, y)
print(grad)
# opt.apply_gradients(zip(grad, model.trainable_variables))

(1, 12)
(1, 3) tf.Tensor([[0.01567648 0.         0.        ]], shape=(1, 3), dtype=float32)
tf.Tensor(-0.005225494, shape=(), dtype=float32)
tf.Tensor(
[[-0.0013156  -0.00511432  0.00331792  0.00055893  0.00248314  0.00168554
   0.00069028  0.00292185 -0.00059704 -0.00135096  0.0026982  -0.00297132]], shape=(1, 12), dtype=float64)


In [185]:
tape.gradient(x2, y)

<tf.Tensor: shape=(1, 12), dtype=float64, numpy=array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])>

In [4]:
optimizer = tf.keras.optimizers.Adam(
        TransformerSchedule(
            d_model=contextnet.dmodel,
            warmup_steps=config.learning_config.optimizer_config.pop("warmup_steps", 10000),
            max_lr=(0.05 / math.sqrt(contextnet.dmodel))
        ),
        **config.learning_config.optimizer_config
    )

contextnet.compile(
    optimizer=optimizer,
    experimental_steps_per_execution=1,
    global_batch_size=1,
    blank=text_featurizer.blank
)

callbacks = [
    tf.keras.callbacks.ModelCheckpoint(**config.learning_config.running_config.checkpoint),
    tf.keras.callbacks.experimental.BackupAndRestore(config.learning_config.running_config.states_dir),
    tf.keras.callbacks.TensorBoard(**config.learning_config.running_config.tensorboard)
]

batch_size = config.learning_config.running_config.batch_size
# eval_data_loader = eval_dataset.create(batch_size)
test_data_loader = test_dataset.create(batch_size)
# value = contextnet.evaluate(eval_data_loader, batch_size=batch_size, callbacks=callbacks)
# print(contextnet.metrics_names,"--------------")
# print(value,"--------------")

def get_weights(net):
    return net.layers[0].get_weights()

  opt = tf.keras.mixed_precision.experimental.LossScaleOptimizer(opt)
Reading /Users/vaibhavsingh/Desktop/TensorFlowASR/scripts/Datasets/test-clean-sanity/transcripts.tsv ...


In [5]:
import os
import io
import abc
import math
import numpy as np
import librosa
import soundfile as sf
import tensorflow as tf
import tensorflow_io as tfio

def load_and_convert_to_wav(path: str) -> tf.Tensor:
    wave, rate = librosa.load(os.path.expanduser(path), sr=None, mono=True)
    print("----------------------------------------")
    print(wave, rate)
    return tf.audio.encode_wav(tf.expand_dims(wave, axis=-1), sample_rate=rate)


def fn(path: bytes):
    return load_and_convert_to_wav(path).numpy()


In [7]:
file = "/Users/vaibhavsingh/Desktop/TensorFlowASR/scripts/Datasets/train-clean-100/LibriSpeech/train-clean-100/26/495/26-495-0000.flac"
# file2 = "/Users/vaibhavsingh/Desktop/TensorFlowASR/scripts/Datasets/train-clean-100/LibriSpeech/train-clean-100/19/198/19-198-0002.flac"


In [22]:
audio = fn(file)
# audio2 = fn(file2)

----------------------------------------
[ 0.00360107  0.00384521 -0.00436401 ...  0.00106812  0.00180054
  0.00146484] 16000


In [9]:
wave, rate = tf.audio.decode_wav(audio, desired_channels=1, desired_samples=-1)
# wave2, rate2 = tf.audio.decode_wav(audio2, desired_channels=1, desired_samples=-1)

In [10]:
print(wave, rate)
# print(wave2, rate2)

tf.Tensor(
[[ 0.00360107]
 [ 0.00384521]
 [-0.00436401]
 ...
 [ 0.00106812]
 [ 0.00180054]
 [ 0.00146484]], shape=(154800, 1), dtype=float32) tf.Tensor(16000, shape=(), dtype=int32)


In [16]:
sample_rate=16000
resampled = tfio.audio.resample(wave, rate_in=tf.cast(rate, dtype=tf.int64), rate_out=sample_rate)
# resampled2 = tfio.audio.resample(wave2, rate_in=tf.cast(rate2, dtype=tf.int64), rate_out=sample_rate)
print(resampled)
# print(resampled2)

tf.Tensor(
[[ 2.08347650e-08]
 [-6.07784827e-08]
 [ 1.01691015e-07]
 ...
 [ 1.85308512e-03]
 [ 1.44618121e-03]
 [ 1.35750882e-03]], shape=(154800, 1), dtype=float32)


In [15]:
signal = tf.reshape(resampled, shape=[-1])
# signal2 = tf.reshape(resampled2, shape=[-1])
print(signal, signal.shape)
# print(signal2)

tf.Tensor(
[ 2.08347650e-08 -6.07784827e-08  1.01691015e-07 ...  1.85308512e-03
  1.44618121e-03  1.35750882e-03], shape=(154800,), dtype=float32) (154800,)


In [17]:
from tensorflow_asr.augmentations.augmentation import Augmentation

In [18]:
features = speech_featurizer.tf_extract(signal)

In [19]:
print(features)


tf.Tensor(
[[[-1.1075792 ]
  [-1.1152742 ]
  [-1.14224   ]
  ...
  [ 0.13340296]
  [-0.13371825]
  [-0.07032747]]

 [[-1.1568089 ]
  [-1.0485059 ]
  [-0.9180523 ]
  ...
  [ 0.18181649]
  [-0.13856299]
  [-0.07530545]]

 [[-1.2461052 ]
  [-1.1882193 ]
  [-1.1071631 ]
  ...
  [ 0.19731618]
  [-0.26973835]
  [-0.41382822]]

 ...

 [[-1.262011  ]
  [-1.0573418 ]
  [-0.8686971 ]
  ...
  [-0.8124411 ]
  [-1.2648714 ]
  [-1.6648448 ]]

 [[-1.3339529 ]
  [-1.1959482 ]
  [-1.0437148 ]
  ...
  [-0.88766176]
  [-1.3892792 ]
  [-1.6648448 ]]

 [[-1.0808375 ]
  [-1.0053201 ]
  [-0.90478504]
  ...
  [-0.922016  ]
  [-1.1421442 ]
  [-1.5924511 ]]], shape=(968, 80, 1), dtype=float32)


In [20]:
input_length = tf.cast(tf.shape(features)[0], tf.int32)
print(input_length)

tf.Tensor(968, shape=(), dtype=int32)


In [23]:
signal = tf.expand_dims(signal, axis=-1)
print(signal)

tf.Tensor(
[[ 2.08347650e-08]
 [-6.07784827e-08]
 [ 1.01691015e-07]
 ...
 [ 1.85308512e-03]
 [ 1.44618121e-03]
 [ 1.35750882e-03]], shape=(154800, 1), dtype=float32)


In [33]:
m = np.random.rand(6,3,4)

In [34]:
m.shape

(6, 3, 4)

k=tf.data.Dataset.from_tensor_slices(m)


In [39]:
k=tf.data.Dataset.from_tensor_slices(m)

In [40]:
obj = ASRSliceDataset(
        speech_featurizer=speech_featurizer,
        text_featurizer=text_featurizer,
        **vars(config.learning_config.test_dataset_config)
    )

In [41]:
obj.read_entries()

Reading /Users/vaibhavsingh/Desktop/TensorFlowASR/scripts/Datasets/test-clean-sanity/transcripts.tsv ...


In [48]:
data = tf.data.Dataset.from_tensor_slices(obj.entries)

In [50]:
obj.entries

array([['/Users/vaibhavsingh/Desktop/TensorFlowASR/scripts/Datasets/test-clean/LibriSpeech/test-clean/61/70970/61-70970-0000.flac',
        '6.075',
        '26 16 22 15 8 1 7 10 21 27 16 16 21 9 1 9 2 5 1 3 6 6 15 1 4 16 14 14 2 15 5 6 5 1 21 16 1 9 10 20 1 14 16 21 9 6 19 28 20 1 4 9 2 14 3 6 19 1 20 16 1 20 16 16 15 1 2 20 1 9 6 1 9 2 5 1 4 16 14 6 1 16 22 21 1 7 19 16 14 1 9 10 20 1 4 16 15 23 6 19 20 6 1 24 10 21 9 1 21 9 6 1 20 18 22 10 19 6'],
       ['/Users/vaibhavsingh/Desktop/TensorFlowASR/scripts/Datasets/test-clean/LibriSpeech/test-clean/61/70968/61-70968-0000.flac',
        '4.905',
        '9 6 1 3 6 8 2 15 1 2 1 4 16 15 7 22 20 6 5 1 4 16 14 17 13 2 10 15 21 1 2 8 2 10 15 20 21 1 21 9 6 1 24 10 27 2 19 5 1 24 9 16 1 9 2 5 1 23 2 15 10 20 9 6 5 1 3 6 9 10 15 5 1 21 9 6 1 4 22 19 21 2 10 15 1 16 15 1 21 9 6 1 13 6 7 21'],
       ['/Users/vaibhavsingh/Desktop/TensorFlowASR/scripts/Datasets/test-clean/LibriSpeech/test-clean/5639/40744/5639-40744-0000.flac',
        '15.77',

In [50]:

    contextnet.load_weights("/Users/vaibhavsingh/Desktop/TensorFlowASR/examples/contextnet/checkpoint/model_trained_01.h5", by_name=True)  
    encoder = contextnet.layers[0]
    print(len(encoder.layers))
    inputs = tf.random.uniform((1,87,80,1), 0, 1, tf.float32)
    inputs_length = tf.random.uniform(shape=(1,))
    predictions = tf.random.uniform(shape=(1,))
    predictions_length = tf.random.uniform(shape=(1,))
    signal = tf.random.uniform(shape = tf.TensorShape((1, 200, 1)))

    abcd = encoder.call_feature_output([inputs, inputs_length, signal])
    adc = encoder.feature_output
    nrm = tf.norm(abcd, axis=1)
    final_node = max(tf.reduce_sum(nrm, axis=0))
    
   



24
inside the call fx of encoder------------------------------ [<tf.Tensor: shape=(1, 87, 80, 1), dtype=float32, numpy=
array([[[[0.36795104],
         [0.40905368],
         [0.42036116],
         ...,
         [0.31243265],
         [0.57353616],
         [0.47037077]],

        [[0.5778127 ],
         [0.6736921 ],
         [0.86984193],
         ...,
         [0.31472683],
         [0.24778426],
         [0.95389485]],

        [[0.47671986],
         [0.36146486],
         [0.5852151 ],
         ...,
         [0.56436086],
         [0.4205544 ],
         [0.25291753]],

        ...,

        [[0.8110529 ],
         [0.37920868],
         [0.02186537],
         ...,
         [0.51123893],
         [0.5723038 ],
         [0.8954637 ]],

        [[0.5424657 ],
         [0.5352715 ],
         [0.9362117 ],
         ...,
         [0.9525564 ],
         [0.24963415],
         [0.17245781]],

        [[0.934829  ],
         [0.04063737],
         [0.6472746 ],
         ...,
         [0.7

Inside convBlock======== tf.Tensor(
[[[ 4.6349182e-06  2.2082093e-08 -2.5364334e-05 ...  2.6263429e-05
    3.1737534e-06 -1.9600342e-05]
  [ 4.6180576e-06  2.8544636e-07 -2.5249819e-05 ...  2.6321755e-05
    3.0860006e-06 -1.9632751e-05]
  [ 4.6449577e-06  2.0806698e-07 -2.5469633e-05 ...  2.6346695e-05
    3.1637264e-06 -1.9656922e-05]
  ...
  [ 4.6429432e-06  2.1128501e-07 -2.5476833e-05 ...  2.6347851e-05
    3.1642983e-06 -1.9666220e-05]
  [ 4.5993797e-06  5.0021919e-07 -2.6327793e-05 ...  2.6346830e-05
    3.2845962e-06 -1.9653342e-05]
  [ 4.6491464e-06  5.4647330e-07 -2.7430542e-05 ...  2.6336518e-05
    3.1655488e-06 -1.9634546e-05]]], shape=(1, 11, 256), dtype=float32)

Inside convBlock======== tf.Tensor(
[[[ 9.5614387e-06  2.4962592e-05  4.7987178e-06 ... -4.3186128e-07
    1.2132849e-06 -1.7407492e-06]
  [ 9.3783738e-06  2.4565237e-05  4.8422658e-06 ... -3.5883679e-07
    1.0737159e-06 -1.7670487e-06]
  [ 9.4059706e-06  2.4302488e-05  4.8252186e-06 ... -4.1609039e-07
    1.02

In [51]:
print(final_node)

tf.Tensor(1.2962365e-05, shape=(), dtype=float32)


In [52]:
inp = tf.random.normal((1,87,80,1), 0, 1, tf.float32)
inp.shape
k = tf.random.normal(shape=(1,))
k.shape

TensorShape([1])

In [57]:
k = tape.gradient(final_node, )

In [58]:
print(k)

None


In [16]:
# d_norm1_tf = tf.norm(tf.reshape(random_vector, (random_vector.shape[0], -1)), axis=1, keepdims=True)[:, :, None]


RuntimeError: tf.gradients is not supported when eager execution is enabled. Use tf.GradientTape instead.

In [ ]:
# tensor= (bs*30*320)
# our features = 30(None)
# 1) output = tf.norm(tensor, axis=1, l2)
# 2) tf.max(tf.sum(output, axis=0))

# This basically gives us the most activated node, reduce it in 1 line [tf.max(tf.sum(tf.norm(tensor, axis=1, l2), axis=0))]


In [92]:
for l in encoder:
    print(l)
    try:
        print(l.convs)
        for k in l.convs:
            print("yo")
            print(k)
    except Exception as e:
        print("error", e)


TypeError: 'ContextNetEncoder' object is not iterable